## Inserção base Completa

In [ ]:
import pandas as pd
import psycopg2 
import math

In [ ]:
amostraP = pd.read_csv('../csv/amostraBHPadronizadaTotal.csv')
amostraP.head()

In [ ]:
database_crawler = "SGM"
user_crawler = "terralab"
host_crawler = "34.23.173.147"
password_crawler = "terralab0705"


conn_crawler = psycopg2.connect(database=database_crawler,
                  user=user_crawler,
                  host=host_crawler,
                  password=password_crawler)

cur_crawler = conn_crawler.cursor()

In [ ]:
def get_max_request_id(cur):
    
    query = """ SELECT MAX(request_id)
                FROM "Request"
                    """
    cur.execute(query)
    maior = cur.fetchall()
    return maior[0][0]

In [ ]:
amostraP = amostraP.fillna( value={'number':'REMOVER','street':	'REMOVER','city':'REMOVER', 'CEP':''})

In [ ]:
batch_size = 50000  # Tamanho do lote

num_batches = math.ceil(len(amostraP) / batch_size)

request_id = get_max_request_id(cur_crawler)

for i in range(num_batches):
    
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(amostraP))

    numero = amostraP['number'].iloc[start_idx:end_idx].to_list()
    street = amostraP['street'].iloc[start_idx:end_idx].to_list()
    city = amostraP['city'].iloc[start_idx:end_idx].to_list()
    district = amostraP['city'].iloc[start_idx:end_idx].to_list()
    postcode = amostraP['CEP'].iloc[start_idx:end_idx].to_list()

    query_insercao = """
        INSERT INTO "Request" (
            request_id,
            full_address,
            number,
            public_place,
            district,
            city,
            zip_code,
            state
        )
        VALUES """
    for i in range(0, len(numero)):
        if numero[i] == 'REMOVER' or street[i] == 'REMOVER' or city[i] == 'REMOVER':
            continue
        else:
            if postcode[i] != '':
                cod_postal = int(postcode[i])
            else:
                cod_postal = postcode[i]
            request_id += 1
            full_address = street[i].replace("'","''") + ', ' + str( numero[i])+ ', ' + city[i].replace("'","''") + ', ' + 'MG'
            query_insercao += """(
                {},
                '{}',
                '{}',
                '{}',
                '{}',
                '{}',
                '{}',
                '{}'),\n""".format(request_id, full_address, 
                                    numero[i], street[i].replace("'","''"),
                                    '',
                                    city[i].replace("'","''"),
                                    cod_postal, 'MG')
        

    query_insercao =query_insercao[:-2] +';'
    #print(query_insercao)

    cur_crawler.execute(query_insercao)
    conn_crawler.commit()

cur_crawler.close()


In [ ]:
# Testando para ver se deu certo

def get_max_adress(cur):
    
    query = """ SELECT *
                FROM "Request"
                ORDER BY request_id DESC
                LIMIT 1
                    """
    cur.execute(query)
    maior = cur.fetchall()
    return maior

maior = get_max_adress(cur_crawler)
print(maior)

## Inserção Amostra

In [1]:
import pandas as pd 

In [2]:
#amostra = pd.read_csv('..\csv\amostraBH5mil.csv')
amostra = pd.read_csv('../csv/baseBHtransformada.csv')
amostra.head()

,Unnamed: 0,IDEND,ID_EDC,ID_LOGRADOURO,SIGLA_TIPO_LOGRADOURO,DESC_TIPO_LOGRADOURO,NOME_LOGRADOURO,NUMERO_IMOVEL,LETRA_IMOVEL,ID_BAIRRO_POPULAR,...,ID_BAIRRO_OFICIAL,NUM_BAIRRO_OFICIAL,TIPO_BAIRRO_OFICIAL,NOME_BAIRRO_OFICIAL,ID_REGIONAL,NOME_REGIONAL,CEP,GEOMETRIA,LATITUDE,LONGITUDE
0,0,05058100277,443458,50581,RUA,RUA,OTAVIO BARRETO,277,NaN,15,...,242,830,Bairro,Boa Vista,3,LESTE,31060390.0,"(-19.89688040126723, -43.9038375951244)",-19.896880,-43.903838
1,1,00789101025,451710,7891,RUA,RUA,BARAO DE COROMANDEL,1025,NaN,481,...,221,200,Bairro,do Barreiro,1,BARREIRO,30640060.0,"(-19.976545248550647, -44.024423385549426)",-19.976545,-44.024423
2,2,01490000623,453157,14900,RUA,RUA,PONTA GROSSA,623,NaN,105,...,247,203,Bairro,Araguaia,1,BARREIRO,30620180.0,"(-19.983122702414487, -43.99861503074346)",-19.983123,-43.998615
3,3,09214600148,466439,92146,RUA,RUA,BUENO DE RIVERA,148,NaN,220,...,247,203,Bairro,Araguaia,1,BARREIRO,30622060.0,"(-19.983744735197362, -43.9911336510454)",-19.983745,-43.991134
4,4,06665100326,461913,66651,RUA,RUA,TAPAIARAS,326,NaN,221,...,247,203,Bairro,Araguaia,1,BARREIRO,30620730.0,"(-19.984679564865772, -43.99589383410702)",-19.984680,-43.995894


In [3]:
amostra.columns

Index(['Unnamed: 0', 'IDEND', 'ID_EDC', 'ID_LOGRADOURO',
       'SIGLA_TIPO_LOGRADOURO', 'DESC_TIPO_LOGRADOURO', 'NOME_LOGRADOURO',
       'NUMERO_IMOVEL', 'LETRA_IMOVEL', 'ID_BAIRRO_POPULAR',
       'NUM_BAIRRO_POPULAR', 'NOME_BAIRRO_POPULAR', 'ID_BAIRRO_OFICIAL',
       'NUM_BAIRRO_OFICIAL', 'TIPO_BAIRRO_OFICIAL', 'NOME_BAIRRO_OFICIAL',
       'ID_REGIONAL', 'NOME_REGIONAL', 'CEP', 'GEOMETRIA', 'LATITUDE',
       'LONGITUDE'],
      dtype='object')

In [4]:
amostraP = amostra[['DESC_TIPO_LOGRADOURO', 'NOME_LOGRADOURO', 'NUMERO_IMOVEL', 'TIPO_BAIRRO_OFICIAL', 'NOME_BAIRRO_OFICIAL', 'CEP','LATITUDE','LONGITUDE']].copy()

amostraP['street'] = amostra['DESC_TIPO_LOGRADOURO'] + ' ' + amostra['NOME_LOGRADOURO']

amostraP.drop(['DESC_TIPO_LOGRADOURO', 'NOME_LOGRADOURO'], axis=1, inplace=True)

amostraP['bairro'] = amostra['TIPO_BAIRRO_OFICIAL'] + ' ' + amostra['NOME_BAIRRO_OFICIAL']

amostraP.drop(['TIPO_BAIRRO_OFICIAL', 'NOME_BAIRRO_OFICIAL'], axis=1, inplace=True)



amostraP.head()


,NUMERO_IMOVEL,CEP,LATITUDE,LONGITUDE,street,bairro
0,277,31060390.0,-19.896880,-43.903838,RUA OTAVIO BARRETO,Bairro Boa Vista
1,1025,30640060.0,-19.976545,-44.024423,RUA BARAO DE COROMANDEL,Bairro do Barreiro
2,623,30620180.0,-19.983123,-43.998615,RUA PONTA GROSSA,Bairro Araguaia
3,148,30622060.0,-19.983745,-43.991134,RUA BUENO DE RIVERA,Bairro Araguaia
4,326,30620730.0,-19.984680,-43.995894,RUA TAPAIARAS,Bairro Araguaia


In [5]:
amostraP['city'] = 'BELO HORIZONTE'
amostraP['state'] = 'MG'

In [6]:
amostraP.rename(columns={'NUMERO_IMOVEL':'number'}, inplace=True)

In [7]:
amostraP

,number,CEP,LATITUDE,LONGITUDE,street,bairro,city,state
0,277,31060390.0,-19.896880,-43.903838,RUA OTAVIO BARRETO,Bairro Boa Vista,BELO HORIZONTE,MG
1,1025,30640060.0,-19.976545,-44.024423,RUA BARAO DE COROMANDEL,Bairro do Barreiro,BELO HORIZONTE,MG
2,623,30620180.0,-19.983123,-43.998615,RUA PONTA GROSSA,Bairro Araguaia,BELO HORIZONTE,MG
3,148,30622060.0,-19.983745,-43.991134,RUA BUENO DE RIVERA,Bairro Araguaia,BELO HORIZONTE,MG
4,326,30620730.0,-19.984680,-43.995894,RUA TAPAIARAS,Bairro Araguaia,BELO HORIZONTE,MG
...,...,...,...,...,...,...,...,...
763224,70,31050750.0,-19.901090,-43.903587,RUA PINDARE,Bairro Mariano de Abreu,BELO HORIZONTE,MG
763225,315,31870750.0,-19.845563,-43.900220,RUA DOM SILVERIO GOMES PIMENTA,Bairro Belmonte,BELO HORIZONTE,MG
763226,275,31870750.0,-19.845562,-43.900221,RUA DOM SILVERIO GOMES PIMENTA,Bairro Belmonte,BELO HORIZONTE,MG
763227,2255,30180118.0,-19.933958,-43.947913,AVENIDA OLEGARIO MACIEL,Seção Urbana Décima,BELO HORIZONTE,MG


In [8]:
amostraP.to_csv('../csv/amostraBHPadronizadaTotal.csv')

### Inserção no Banco

In [ ]:
import pandas as pd

In [ ]:
amostraP = pd.read_csv('..\csv\amostraBHPadronizada5mil.csv')

In [ ]:
amostraP = amostraP[['number', 'CEP', 'street', 'bairro', 'city', 'state']]

In [ ]:
amostraP

In [ ]:
import psycopg2
import io

In [ ]:
database_crawler = "SGM"
user_crawler = "terralab"
host_crawler = "34.23.173.147"
password_crawler = "terralab0705"


conn_crawler = psycopg2.connect(database=database_crawler,
                  user=user_crawler,
                  host=host_crawler,
                  password=password_crawler)

cur_crawler = conn_crawler.cursor()

In [ ]:
def get_max_request_id(cur):
    
    query = """ SELECT MAX(request_id)
                FROM "Request"
                    """
    cur.execute(query)
    maior = cur.fetchall()
    return maior[0][0]

In [ ]:
amostraP = amostraP.fillna( value={'number':'REMOVER','street':	'REMOVER','city':'REMOVER', 'CEP':''})

In [ ]:
amostraP.to_csv('../csv/amostraBHPadronizadaTotal.csv')

In [ ]:
amostraP.columns

In [ ]:
numero = amostraP['number'].to_list()
street = amostraP['street'].to_list()
city = amostraP['city'].to_list()
district = amostraP['city'].to_list()
postcode = amostraP['CEP'].to_list()
bairro = amostraP['bairro'].to_list()

In [ ]:
request_id = get_max_request_id(cur_crawler)

In [ ]:
request_id

In [ ]:
query_insercao = """
    INSERT INTO "Request" (
        request_id,
        full_address,
        number,
        public_place,
        district,
        city,
        zip_code,
        state
    )
    VALUES """
for i in range(0, len(numero)):
    if numero[i] == 'REMOVER' or street[i] == 'REMOVER' or city[i] == 'REMOVER':
        continue
    else:
        if postcode[i] != '':
            cod_postal = int(postcode[i])
        else:
            cod_postal = postcode[i]
        request_id += 1
        full_address = street[i].replace("'","''") + ', ' + str( numero[i])+ ', ' + city[i].replace("'","''") + ', ' + 'MG'
        query_insercao += """(
            {},
            '{}',
            '{}',
            '{}',
            '{}',
            '{}',
            '{}',
            '{}'),\n""".format(request_id, full_address, 
                                numero[i], street[i].replace("'","''"),
                                '',
                                city[i].replace("'","''"),
                                cod_postal, 'MG')
    

query_insercao =query_insercao[:-2] +';'

In [ ]:
print(query_insercao)

In [ ]:
cur_crawler.execute(query_insercao)
cur_crawler.close()
conn_crawler.commit()

In [ ]:
request_id

In [ ]:
request_id = get_max_request_id(cur_crawler)
request_id

In [ ]:

def get_max_adress(cur):
    
    query = """ SELECT *
                FROM "Request"
                ORDER BY request_id DESC
                LIMIT 1
                    """
    cur.execute(query)
    maior = cur.fetchall()
    return maior

#query = 'SELECT * FROM Request  AS r ORDER BY r.request_id DESC LIMIT 1'


In [ ]:
ultimo = get_max_request_id(cur_crawler)

In [ ]:
ultimo

## Experimentos diversos 

### Obs: Modifiquei esse código inúmeras vezes, o de cima é mais confiável

In [ ]:
#import pandas as pd
#import psycopg2

request_id = get_max_request_id(cur_crawler)
query_insercao = """
    INSERT INTO "Request" (
        request_id,
        full_address,
        number,
        public_place,
        district,
        city,
        zip_code,
        state
    )
    VALUES """
for i in range(0, len(numero)):
    if numero[i] == 'REMOVER' or street[i] == 'REMOVER' or city[i] == 'REMOVER':
        continue
    else:
        if postcode[i] != '':
            cod_postal = int(postcode[i])
        else:
            cod_postal = postcode[i]
        request_id += 1
        full_address = street[i].replace("'","''") +  ', ' + str( numero[i]) + ', ' + bairro[i].replace("'","''") + ', ' + city[i].replace("'","''") + ', ' + 'MG'
        query_insercao += """(
            {},
            '{}',
            '{}',
            '{}',
            '{}',
            '{}',
            '{}',
            '{}'),\n""".format(request_id, full_address, 
                                numero[i], street[i].replace("'","''"),
                                '',
                                city[i].replace("'","''"),
                                cod_postal, 'MG')
    

query_insercao =query_insercao[:-2] +';'

print(query_insercao)

#bairro[i].replace("'","''")

In [ ]:
import pandas as pd
import psycopg2
database_crawler = "SGM"
user_crawler = "terralab"
host_crawler = "34.23.173.147"
password_crawler = "terralab0705"


conn_crawler = psycopg2.connect(database=database_crawler,
                  user=user_crawler,
                  host=host_crawler,
                  password=password_crawler)

cur_crawler = conn_crawler.cursor()

In [ ]:
cur_crawler.execute(query_insercao)
ultimo = get_max_adress(cur_crawler)
print(ultimo)
cur_crawler.close()
conn_crawler.commit()

In [ ]:
ultimo = get_max_adress(cur_crawler)

In [ ]:
ultimo